In [14]:
import pandas as pd
import numpy as np
import tensorflow as tf
from transformers import AutoModel , AutoImageProcessor , AutoTokenizer 
from datasets import load_dataset , Dataset
from datasets import Image as dImage
from PIL import Image
import torch
from torch import nn

torch.cuda.empty_cache()

# import os
# os.environ['TF_GPU_ALLOCATOR']='cuda_malloc_async'

2024-05-16 12:42:02.774786: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-16 12:42:03.349117: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/home/user/miniconda3/envs/python38/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
df_acc = pd.read_pickle("./vqa_v2_acc.pkl")
df_acc.head()

,question,question_type,question_id,image_id,answer_type,label
0,What is this photo taken looking through?,what is this,458752000,COCO_train2014_000000458752.jpg,other,"{'ids': ['net', 'netting', 'mesh'], 'weights':..."
1,What position is this man playing?,what,458752001,COCO_train2014_000000458752.jpg,other,"{'ids': ['pitcher', 'catcher'], 'weights': [0...."
2,What color is the players shirt?,what color is the,458752002,COCO_train2014_000000458752.jpg,other,"{'ids': ['orange'], 'weights': [1.0]}"
3,Is this man a professional baseball player?,is this,458752003,COCO_train2014_000000458752.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.94, 0.06]}"
4,What color is the snow?,what color is the,262146000,COCO_train2014_000000262146.jpg,other,"{'ids': ['white'], 'weights': [1.0]}"


In [29]:
np.array(Image.open("./train2014_3d/" + df_acc["image_id"][0])).shape

(480, 640, 3)

In [105]:
# SAMPLES = 100
# df = df_acc[:SAMPLES]

df = df_acc.groupby('question_type').sample(n=200 , replace=True)
df = df.reset_index(drop=True)
SAMPLES = len(df)

print("Number of samples are : " , SAMPLES)

df.head()

Number of samples are :  13000


,question,question_type,question_id,image_id,answer_type,label
0,Are all the men using headphones?,are,448439000,COCO_train2014_000000448439.jpg,yes/no,"{'ids': ['no'], 'weights': [1.0]}"
1,Are any of these ties tied?,are,21674004,COCO_train2014_000000021674.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.79, 0.21]}"
2,Are all of these adults?,are,532439000,COCO_train2014_000000532439.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.8, 0.2]}"
3,Are both doors open on the icebox?,are,341218001,COCO_train2014_000000341218.jpg,yes/no,"{'ids': ['yes'], 'weights': [1.0]}"
4,Are those mountains in the distance?,are,23249002,COCO_train2014_000000023249.jpg,yes/no,"{'ids': ['yes', 'no'], 'weights': [0.95, 0.05]}"


Creating Labels

In [106]:
import itertools

labels = [item['ids'] for item in df['label'].to_list()]
flattened_labels = list(itertools.chain(*labels))
unique_labels = list(set(flattened_labels))

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()} 

In [107]:
label2id

{'hard to read': 0,
 'over shoulder': 1,
 '$4.20': 2,
 'near people': 3,
 'raspberry tart': 4,
 'cutlery': 5,
 'pudding': 6,
 'guitar': 7,
 'pizza pan': 8,
 'fake tree': 9,
 '2:43': 10,
 'oldest man': 11,
 'humming bird and bird are stop action photo': 12,
 'asian': 13,
 'lab': 14,
 'pound symbol': 15,
 'under wave': 16,
 'buildings': 17,
 "in girl's mouth": 18,
 'acoustic': 19,
 'in': 20,
 'cleanse hands': 21,
 'yacht': 22,
 'blue bird': 23,
 'pavement': 24,
 'it is tennis racquet in her hand': 25,
 'seed packet': 26,
 'on top of train': 27,
 'women, bags': 28,
 'because of heat': 29,
 'slow': 30,
 '1 and 2': 31,
 'pants and dress': 32,
 'valparaiso': 33,
 '11:55': 34,
 'fall': 35,
 'carmel': 36,
 '12:30': 37,
 '1 hour': 38,
 'its black and white': 39,
 'bank': 40,
 'outhouse': 41,
 'succotash': 42,
 'construction site': 43,
 'hurst grove, street pride': 44,
 'elderly gentleman': 45,
 'accents': 46,
 'for storage': 47,
 'hp': 48,
 'pull': 49,
 'bicycle proceed': 50,
 'stairway': 51,
 

In [109]:
NUM_CLASSES = len(id2label)
print("Number of Classes : " , NUM_CLASSES)

Number of Classes :  12251


In [110]:
n = SAMPLES
rf = 10 ** (len(str(n))-1)
r = str(int(np.round(n / rf) * rf))

r = r[:-3] + r[-3:].replace("000" , "k")

pd.to_pickle(df , f"./vqa_v2_sampled_{r}.pkl")

### Evaluate Existing Dataset

In [121]:
ds = pd.read_pickle("vqa_v2_sampled_10k.pkl")
# # ds = df_acc.groupby('question_type').sample(n=200 , replace=True)
# # ds = ds.reset_index(drop=True)

# ds = df_acc[:3]

SAMPLES = len(ds)
print("Number of samples are : " , SAMPLES)

labels = [item['ids'] for item in ds['label'].to_list()]
flattened_labels = list(itertools.chain(*labels))
unique_labels = list(set(flattened_labels))

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()} 

NUM_CLASSES = len(id2label)
print("Number of Classes : " , NUM_CLASSES)

label2id

Number of samples are :  13000
Number of Classes :  12251


{'hard to read': 0,
 'over shoulder': 1,
 '$4.20': 2,
 'near people': 3,
 'raspberry tart': 4,
 'cutlery': 5,
 'pudding': 6,
 'guitar': 7,
 'pizza pan': 8,
 'fake tree': 9,
 '2:43': 10,
 'oldest man': 11,
 'humming bird and bird are stop action photo': 12,
 'asian': 13,
 'lab': 14,
 'pound symbol': 15,
 'under wave': 16,
 'buildings': 17,
 "in girl's mouth": 18,
 'acoustic': 19,
 'in': 20,
 'cleanse hands': 21,
 'yacht': 22,
 'blue bird': 23,
 'pavement': 24,
 'it is tennis racquet in her hand': 25,
 'seed packet': 26,
 'on top of train': 27,
 'women, bags': 28,
 'because of heat': 29,
 'slow': 30,
 '1 and 2': 31,
 'pants and dress': 32,
 'valparaiso': 33,
 '11:55': 34,
 'fall': 35,
 'carmel': 36,
 '12:30': 37,
 '1 hour': 38,
 'its black and white': 39,
 'bank': 40,
 'outhouse': 41,
 'succotash': 42,
 'construction site': 43,
 'hurst grove, street pride': 44,
 'elderly gentleman': 45,
 'accents': 46,
 'for storage': 47,
 'hp': 48,
 'pull': 49,
 'bicycle proceed': 50,
 'stairway': 51,
 

In [122]:
ds["label"].tolist()

[{'ids': ['no'], 'weights': [1.0]},
 {'ids': ['yes', 'no'], 'weights': [0.79, 0.21]},
 {'ids': ['yes', 'no'], 'weights': [0.8, 0.2]},
 {'ids': ['yes'], 'weights': [1.0]},
 {'ids': ['yes', 'no'], 'weights': [0.95, 0.05]},
 {'ids': ['no', 'yes'], 'weights': [0.94, 0.06]},
 {'ids': ['yes'], 'weights': [1.0]},
 {'ids': ['yes'], 'weights': [1.0]},
 {'ids': ['yes'], 'weights': [1.0]},
 {'ids': ['no', 'yes'], 'weights': [0.82, 0.18]},
 {'ids': ['yes', 'no'], 'weights': [0.8, 0.2]},
 {'ids': ['yes'], 'weights': [1.0]},
 {'ids': ['no', 'yes'], 'weights': [0.8, 0.2]},
 {'ids': ['no', 'no windows', 'yes'], 'weights': [0.82, 0.12, 0.06]},
 {'ids': ['yes'], 'weights': [1.0]},
 {'ids': ['no'], 'weights': [1.0]},
 {'ids': ['no'], 'weights': [1.0]},
 {'ids': ['no'], 'weights': [1.0]},
 {'ids': ['no'], 'weights': [1.0]},
 {'ids': ['yes', 'no'], 'weights': [0.42, 0.58]},
 {'ids': ['yes'], 'weights': [1.0]},
 {'ids': ['no'], 'weights': [1.0]},
 {'ids': ['no', 'yes'], 'weights': [0.9, 0.1]},
 {'ids': ['ye

In [124]:
# h = []
# for i in ds["label"]:
#     r = [0 for i in id2label]
#     for t , w in zip(i["ids"] , i["weights"]):
#         r[label2id[t]] = w
#     h.append(r)
# h

Evaluating Image frequencies in Dataset

In [127]:
len(ds["image_id"].unique()) , len(ds)

(11189, 13000)

In [130]:
vc = ds.image_id.value_counts()

In [141]:
vc[vc > 4]

image_id
COCO_train2014_000000150435.jpg    12
COCO_train2014_000000334098.jpg     8
COCO_train2014_000000276711.jpg     7
COCO_train2014_000000174680.jpg     6
COCO_train2014_000000211051.jpg     5
COCO_train2014_000000407465.jpg     5
COCO_train2014_000000265622.jpg     5
COCO_train2014_000000474075.jpg     5
COCO_train2014_000000479372.jpg     5
COCO_train2014_000000351972.jpg     5
COCO_train2014_000000445074.jpg     5
COCO_train2014_000000243354.jpg     5
COCO_train2014_000000098494.jpg     5
COCO_train2014_000000383968.jpg     5
COCO_train2014_000000490400.jpg     5
Name: count, dtype: int64

Number of Unique Images in Dataset

### Creating Yes/No dataset
Creating a dataset with only yes / no answers

In [203]:
ds = df_acc.loc[df_acc["answer_type"] == "yes/no"]
ds_yes_no_cols = ds['label'].apply(pd.Series)

In [287]:
df_yes_no = pd.concat([ds , ds_yes_no_cols] , axis=1)
df_yes_no["ids"] = df_yes_no["ids"].apply(lambda x: x[0])
df_yes_no["weights"] = df_yes_no["weights"].apply(lambda x: x[0])
df_yes_no = df_yes_no.loc[df_yes_no["weights"] > 0.5]
df_yes_no = df_yes_no.loc[(df_yes_no["ids"] == "yes") | (df_yes_no["ids"] == "no")]
df_yes_no["ids"] = df_yes_no["ids"].apply(lambda x: 1 if x=="yes" else 0)

df_yes_no = df_yes_no.drop(["label"] , axis=1)

df_yes_no = df_yes_no.rename(columns={'ids':'label'})

print("Total number of samples : " , len(df_yes_no))
df_yes_no.head()


Total number of samples :  148090


,question,question_type,question_id,image_id,answer_type,label,weights
3,Is this man a professional baseball player?,is this,458752003,COCO_train2014_000000458752.jpg,yes/no,1,0.94
8,Is the dog waiting?,is the,524291001,COCO_train2014_000000524291.jpg,yes/no,1,1.00
10,Is the sky blue?,is the,393221000,COCO_train2014_000000393221.jpg,yes/no,1,1.00
11,Is there snow on the mountains?,is there,393221001,COCO_train2014_000000393221.jpg,yes/no,1,1.00
13,Is the window open?,is the,393223000,COCO_train2014_000000393223.jpg,yes/no,1,0.82


This already covers about 1/4th of the dataset

In [285]:
df = df_yes_no.groupby('question_type').sample(n=4000 , replace=True)
df = df.reset_index(drop=True)
SAMPLES = len(df)

n = SAMPLES
rf = 10 ** (len(str(n))-1)
r = str(int(np.round(n / rf) * rf))
r = r[:-3] + r[-3:].replace("000" , "k")
pd.to_pickle(df , f"./vqa_v2_yes_no_{r}.pkl")

print("Number of samples : " , SAMPLES)
df.head()

Number of samples :  140000


,question,question_type,question_id,image_id,answer_type,label,weights
0,Are all the buildings white?,are,489914000,COCO_train2014_000000489914.jpg,yes/no,1,0.78
1,Are those birds seen in the sky?,are,428503000,COCO_train2014_000000428503.jpg,yes/no,0,0.79
2,Are all the animals getting along?,are,73104004,COCO_train2014_000000073104.jpg,yes/no,1,1.00
3,Are all the people in this scene standing?,are,152096001,COCO_train2014_000000152096.jpg,yes/no,0,0.80
4,Are objects in the picture vases?,are,342665000,COCO_train2014_000000342665.jpg,yes/no,1,0.74


In [286]:
ans = df["label"]
ans[ans == 1].count() , ans.count()

(63799, 140000)

### Creating Clipped Dataset
Limiting number of outputs to 1000

In [281]:
from collections import Counter

answers = Counter()

for annot in df_acc["label"]:
    for answer in annot['ids']:
        answers[answer] += 1

most_common_answers = answers.most_common(1000)
most_common_answers = {i:j for (i,j) in most_common_answers}
most_common_answers

{'yes': 124749,
 'no': 122172,
 '2': 21902,
 '1': 20144,
 'white': 14474,
 '3': 14326,
 '4': 11258,
 '0': 9576,
 'black': 9446,
 'blue': 8912,
 'red': 8599,
 'brown': 8061,
 '5': 7654,
 'green': 6743,
 'nothing': 6350,
 'gray': 6271,
 '6': 5898,
 'none': 5492,
 'yellow': 5338,
 'unknown': 4222,
 '7': 4114,
 '8': 3825,
 'orange': 3657,
 'left': 3654,
 '10': 3544,
 'right': 3543,
 'maybe': 3531,
 'tan': 2842,
 'wood': 2715,
 '12': 2408,
 '9': 2396,
 'many': 2334,
 'pink': 2314,
 'water': 2267,
 'baseball': 2261,
 'man': 2242,
 'food': 2180,
 'tennis': 2112,
 'silver': 2105,
 'frisbee': 2097,
 'grass': 2051,
 'not sure': 2037,
 'black and white': 1978,
 '20': 1915,
 "don't know": 1824,
 'beige': 1774,
 "can't tell": 1734,
 'table': 1730,
 'pizza': 1722,
 'outside': 1693,
 '15': 1689,
 'standing': 1573,
 'kitchen': 1573,
 'bathroom': 1521,
 'dog': 1514,
 'skiing': 1467,
 '11': 1439,
 'lot': 1429,
 'phone': 1427,
 'cat': 1421,
 'kite': 1370,
 'not possible': 1313,
 'ground': 1295,
 'skatebo

In [282]:
print("These answers constitute " , round(sum(most_common_answers.values()) / sum(answers.values()) * 100 , 0) , "% of dataset")

These answers constitute  64.0 % of dataset


In [296]:
df = pd.read_pickle("./vqa_v2_acc.pkl")

def top_label(x):
    flag = True
    for i in x['ids']:
        if i not in most_common_answers: flag = False
    return x if flag else 'NONE'

df["label"] = df["label"].apply(top_label)
df = df.loc[df["label"] != 'NONE']
len(df) / len(df_acc)

0.6399966648413433

In [297]:
labels = [item['ids'] for item in df['label'].to_list()]
flattened_labels = list(itertools.chain(*labels))
unique_labels = list(set(flattened_labels))

label2id = {label: idx for idx, label in enumerate(unique_labels)}
id2label = {idx: label for label, idx in label2id.items()} 

NUM_CLASSES = len(id2label)
print("Number of Classes : " , NUM_CLASSES)

Number of Classes :  1000


In [306]:
df['question_type'].value_counts().head(20)

question_type
how many              38773
is the                31900
what color is the     19003
is this               15223
is this a             15001
are the                9757
what                   9711
is there a             9553
none of the above      9324
what is the            7441
is it                  6935
is there               6171
does the               5714
is                     5691
are there              5540
are these              5319
is the man             4644
are                    4631
does this              4121
what color are the     4060
Name: count, dtype: int64

In [321]:
ds = df.groupby('question_type').sample(n=440 , replace=True)
ds = ds.reset_index(drop=True)

SAMPLES = len(ds)

print("Number of samples are : " , SAMPLES)

Number of samples are :  28600


In [320]:
n = SAMPLES
rf = 10 ** (len(str(n))-1)
r = str(int(np.round(n / rf) * rf))

r = r[:-3] + r[-3:].replace("000" , "k")

pd.to_pickle(ds , f"./vqa_v2_sampled_{r}_clipped.pkl")